In [ ]:
import tensorflow as tf
import numpy as np
 
#编写TextCNN类
class TextCNN:
    def __init__(self, filter_sizes,num_filters,num_classes, learning_rate, batch_size, decay_steps, decay_rate,sequence_length,vocab_size,embed_size,
                 is_training,initializer=tf.random_normal_initializer(stddev=0.1),multi_label_flag=False,clip_gradients=5.0,decay_rate_big=0.50):
        """init all hyperparameter here"""
        # 设置参数
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.sequence_length=sequence_length
        self.vocab_size=vocab_size
        self.embed_size=embed_size
        self.is_training=is_training
        self.learning_rate = tf.Variable(learning_rate, trainable=False, name="learning_rate")#ADD learning_rate
        self.learning_rate_decay_half_op = tf.assign(self.learning_rate, self.learning_rate * decay_rate_big)
        self.filter_sizes=filter_sizes # it is a list of int. e.g. [3,4,5]
        self.num_filters=num_filters
        self.initializer=initializer
        self.num_filters_total=self.num_filters * len(filter_sizes) #how many filters totally.
        self.multi_label_flag=multi_label_flag
        self.clip_gradients = clip_gradients
 
        # 设置标签参数
        self.input_x = tf.placeholder(tf.int32, [None, self.sequence_length], name="input_x")  # X
        self.input_y = tf.placeholder(tf.int32, [None,],name="input_y")  # y:[None,num_classes]
        self.input_y_multilabel = tf.placeholder(tf.float32,[None,self.num_classes], name="input_y_multilabel")  # y:[None,num_classes]. this is for multi-label classification only.
        self.dropout_keep_prob=tf.placeholder(tf.float32,name="dropout_keep_prob")
 
        self.global_step = tf.Variable(0, trainable=False, name="Global_Step")
        self.epoch_step=tf.Variable(0,trainable=False,name="Epoch_Step")
        self.epoch_increment=tf.assign(self.epoch_step,tf.add(self.epoch_step,tf.constant(1)))
        self.decay_steps, self.decay_rate = decay_steps, decay_rate
 
        self.instantiate_weights()
        self.logits = self.inference()
        if not is_training:
            return
        if multi_label_flag:
            print("going to use multi label loss.")
            self.loss_val = self.loss_multilabel()
        else:
            print("going to use single label loss.")
            self.loss_val = self.loss()
        self.train_op = self.train()
        self.predictions = tf.argmax(self.logits, 1, name="predictions")  # shape:[None,]
 
        if not self.multi_label_flag:
            correct_prediction = tf.equal(tf.cast(self.predictions,tf.int32), self.input_y) #tf.argmax(self.logits, 1)-->[batch_size]
            self.accuracy =tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="Accuracy") # shape=()
        else:
            self.accuracy = tf.constant(0.5) #fuke accuracy. (you can calcuate accuracy outside of graph using method calculate_accuracy(...) in train.py)
 
     #设置权重
    def instantiate_weights(self):
        """define all weights here"""
        with tf.name_scope("embedding"): # embedding matrix
            self.Embedding = tf.get_variable("Embedding",shape=[self.vocab_size, self.embed_size],initializer=self.initializer) 
            self.W_projection = tf.get_variable("W_projection",shape=[self.num_filters_total, self.num_classes],initializer=self.initializer) 
            self.b_projection = tf.get_variable("b_projection",shape=[self.num_classes])       
 
    def inference(self):
        """main computation graph here: 1.embedding-->2.average-->3.linear classifier"""
        # 获得文本长度等信息
        self.embedded_words = tf.nn.embedding_lookup(self.Embedding,self.input_x)
        self.sentence_embeddings_expanded=tf.expand_dims(self.embedded_words,-1) 
 
        pooled_outputs = []
        for i,filter_size in enumerate(self.filter_sizes):
            with tf.name_scope("convolution-pooling-%s" %filter_size):
               #过滤器设置
                filter = tf.get_variable("filter-%s"%filter_size,[filter_size,self.embed_size,1,self.num_filters],initializer=self.initializer)
                conv=tf.nn.conv2d(self.sentence_embeddings_expanded, filter, strides=[1,1,1,1], padding="VALID",name="conv") 
                b=tf.get_variable("b-%s"%filter_size,[self.num_filters]) 
                h=tf.nn.relu(tf.nn.bias_add(conv,b),"relu") 
            
                pooled = tf.nn.max_pool(h, ksize=[1,self.sequence_length-filter_size+1,1,1], strides=[1,1,1,1], padding='VALID',name="pool")
                pooled_outputs.append(pooled)
        self.h_pool=tf.concat(pooled_outputs,3) 
        self.h_pool_flat=tf.reshape(self.h_pool,[-1,self.num_filters_total]) 
 
        #过滤层
        with tf.name_scope("dropout"):
            self.h_drop  =  tf.nn.dropout(self.h_pool_flat,keep_prob=self.dropout_keep_prob) 
        with tf.name_scope("output"):
            logits = tf.matmul(self.h_drop,self.W_projection) + self.b_projection 
        return logits
 #损失函数
    def loss(self,l2_lambda=0.0001):
        with tf.name_scope("loss"):
            
            losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.input_y, logits=self.logits);
            loss=tf.reduce_mean(losses)
            l2_losses = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'bias' not in v.name]) * l2_lambda
            loss=loss+l2_losses
        return loss
 
    def loss_multilabel(self,l2_lambda=0.00001): 
        with tf.name_scope("loss"):
          
            losses = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.input_y_multilabel, logits=self.logits);
            print("sigmoid_cross_entropy_with_logits.losses:",losses) 
            losses=tf.reduce_sum(losses,axis=1)
            loss=tf.reduce_mean(losses)         
            l2_losses = tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'bias' not in v.name]) * l2_lambda
            loss=loss+l2_losses
        return loss
 #训练集处理
    def train(self):
        """based on the loss, use SGD to update parameter"""
        learning_rate = tf.train.exponential_decay(self.learning_rate, self.global_step, self.decay_steps,self.decay_rate, staircase=True)
        train_op = tf.contrib.layers.optimize_loss(self.loss_val, global_step=self.global_step,learning_rate=learning_rate, optimizer="Adam",clip_gradients=self.clip_gradients)
        return train_op
 
 


In [ ]:
import numpy as np
import pandas as pd
import nltk
from collections import Counter
MAX_FEATURES = 1500
MAX_SENTENCE_LENGTH = 100
 
#设置初始参数
lr = 0.001
training_iters = 100000
batch_size = 127
vocab_size = MAX_FEATURES
embedding_size = 100
n_inputs = embedding_size  # 最小数据输入维度
n_steps = MAX_SENTENCE_LENGTH  # 步长
n_hidden_units = 128  
 
def get_sentiment_data():
    df_sentiment = pd.read_csv('sentiment.csv', encoding='utf-8')
    sentenses = df_sentiment['sentence'].values
    sentenses = [s.lower() for s in sentenses]
    wordlist_sentence = [nltk.word_tokenize(s) for s in sentenses]
    ws = []
    for wordlist in wordlist_sentence:
        ws.extend(wordlist)
    word_counter = Counter(ws)
    mc = word_counter.most_common(100)

    vocab_size = min(MAX_FEATURES, len(word_counter)) + 2
    word2index = {x[0]: i + 2 for i, x in
                  enumerate(word_counter.most_common(MAX_FEATURES))}
    word2index["PAD"] = 0
    word2index["UNK"] = 1
    index2word = {v: k for k, v in word2index.items()}
    res = []
    print('iterrows')
    for line in df_sentiment.iterrows():
        label, sentence = str(line[1]['label']), line[1]['sentence']
        words = nltk.word_tokenize(sentence.lower())
        for word in words:
            if word in word2index.keys():
                seqs1.append(word2index[word])
            else:
                seqs1.append(word2index["UNK"])
        if MAX_SENTENCE_LENGTH < len(seqs1):
            print('unexpected length of padding', len(padding), padding)
            continue
        padding = [0] * (MAX_SENTENCE_LENGTH - len(seqs1))
        padding.extend(seqs1)
        if len(padding) != MAX_SENTENCE_LENGTH:
            print('unexpected length of padding', len(padding), padding)
        if label == '0':
            res.append([1, padding])
        if label == '1':
            res.append([0, padding])
    return res,vocab_size
 

In [ ]:
#test函数
def test():
    num_classes=2
    learning_rate=lr
    batch_size=100
    decay_steps=1000
    decay_rate=0.9

    is_training=True
    dropout_keep_prob=1 
    filter_sizes=[3,4]
    num_filters=128
 
    sentiment,vocab_size = get_sentiment_data()
    input_x = [x for y,x in sentiment]
    Ys = [y for y,x in sentiment]
    from sklearn.model_selection import train_test_split
    Xtrain, Xtest, ytrain, ytest = train_test_split(input_x, Ys, test_size=0.1, random_state=42, stratify=Ys)
 
    textRNN=TextCNN(filter_sizes,num_filters,num_classes, learning_rate, batch_size, decay_steps, decay_rate,MAX_SENTENCE_LENGTH,vocab_size,embedding_size,is_training)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(100):
            print('\n--------TRAIN--------:  No.', (i ) * batch_size)
            if (i+1)*batch_size >len(Xtrain):
                print('test No.',(i+1)*batch_size )
                break
            input_x = Xtrain[i*batch_size:(i+1)*batch_size]
            input_y = ytrain[i*batch_size:(i+1)*batch_size]
            loss,acc,predict,W_projection_value,_=sess.run(
                [textRNN.loss_val,textRNN.accuracy,textRNN.predictions,textRNN.W_projection,textRNN.train_op],
                feed_dict={textRNN.input_x:input_x,textRNN.input_y:input_y,textRNN.dropout_keep_prob:dropout_keep_prob})
            print("loss:",loss,"acc:",acc)


In [ ]:
test()